In [1]:
from google.colab import drive
drive.mount('/content/drive')

!ls drive/MyDrive/earth_engine/ee-manmeet20singh15-wbis-fab7f1ca35e0.json
!ls /content/drive/MyDrive/ndui

Mounted at /content/drive
drive/MyDrive/earth_engine/ee-manmeet20singh15-wbis-fab7f1ca35e0.json
best_model_albuquerque.pth  best_model_shanghai.pth	  dmsp_1992_2022_tokyo.nc
best_model_berlin.pth	    best_model_tokyo.pth	  dmsp_2012_austin.nc
best_model_cairo.pth	    best_model_washington_dc.pth  dmsp_swinIR_1992_2022_delhi.nc
best_model_columbus.pth     cur_model_epoch0.h5		  dmsp_swinIR_1992_2022_dubai.nc
best_model_delhi.pth	    cur_model_epoch1.h5		  dmsp_swinIR_1992_2022_hongkong.nc
best_model_denver.pth	    cur_model_epoch2.h5		  dmsp_swinIR_1992_2022_london.nc
best_model_dhaka.pth	    cur_model_epoch3.h5		  dmsp_swinIR_1992_2022_melbourne.nc
best_model_dubai.pth	    cur_model_epoch4.h5		  Guangzhou_studyarea.shp
best_model_hongkong.pth     cur_model_epoch5.h5		  Guangzhou_studyarea.shx
best_model_kansas_city.pth  cur_model_epoch6.h5		  model_MAE_depth_4_per_level_3.h5
best_model_la.pth	    cur_model_epoch7.h5		  ndui_new_york.nc
best_model_london.pth	    cur_model_epoch8.h5		  r

In [2]:
import ee
import os

# Set the path to the service account key file
service_account = 'editor@ee-manmeet20singh15-wbis.iam.gserviceaccount.com'
key_file = 'drive/MyDrive/earth_engine/ee-manmeet20singh15-wbis-fab7f1ca35e0.json'

# Use the service account for authentication
credentials = ee.ServiceAccountCredentials(service_account, key_file)
ee.Initialize(credentials)

!pip install timm
!pip install wxee

import ee
import wxee
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms import Compose, ToTensor, Resize
from PIL import Image
import os
import numpy as np
import xarray as xr
import pandas as pd

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 656.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.3 MB/s eta 0:00:00


In [3]:
# Data for one city (e.g., 'delhi')
city = ['delhi']
lat_ = [28.7041]
lon_ = [77.1025]

# For a single year (e.g., 2000)
year_ = 2022
year = str(year_)

# Creating DataFrame for one city
df = pd.DataFrame({'Name': city, 'lat': lat_, 'lon': lon_})

# Define the AOI for the city
aoi_city = []
for i in range(len(df.lat)):
  lats, late = df.lat[i]-0.2, df.lat[i]+0.2
  lons, lone = df.lon[i]-0.2, df.lon[i]+0.2
  aoi = ee.Geometry.Polygon(
        [[[lons, lats],
          [lone, lats],
          [lone, late],
          [lons, late]]])
  aoi_city.append(aoi)
aoi_city[0]



# Load the Landsat 7 dataset for the year
L7 = ee.ImageCollection("LANDSAT/LE07/C02/T1_TOA").filterDate(year+'-01-01', year+'-12-31')

def fun4(img):
    bad1 = img.select('B1').eq(0.0)
    bad2 = img.select('B2').eq(0.0)
    bad3 = img.select('B3').eq(0.0)
    bad4 = img.select('B4').eq(0.0)
    bad5 = img.select('B5').eq(0.0)
    bad7 = img.select('B7').eq(0.0)
    mask = img.mask().And(bad1.Or(bad2).Or(bad3).Or(bad4).Or(bad5).Or(bad7).Not())
    masked = img.mask(mask)
    ndvi = masked.normalizedDifference(["B4", "B3"])
    return ndvi

NDVIs = L7.map(fun4)

# Generate NDVI statistics for the year
Mean_NDVI = NDVIs.median()
Max_NDVI = NDVIs.max()
Min_NDVI = NDVIs.min()

mosaic = Mean_NDVI.where(Max_NDVI.gt(0.4), Max_NDVI)
mosaic = mosaic.where(Min_NDVI.lt(-0.2), Min_NDVI)

mosaic = mosaic.set('system:time_start', 0)
ds_ndvi_c = mosaic.wx.to_xarray(region=aoi_city[0].bounds(), scale=30)
ds_ndvi_c_ = ds_ndvi_c.nd.values[0,:,:]

# Load the DMSP dataset and interpolate for the same year
ds_dmsp_1992_2022_city = xr.open_dataset('/content/drive/MyDrive/ndui/dmsp_swinIR_1992_2022_delhi.nc')
ds_dmsp_interp_c = ds_dmsp_1992_2022_city.sel(time=slice(year, year)).interp(lon=ds_ndvi_c.x, lat=ds_ndvi_c.y).dmsp.values[0,:,:]/63.0

# Calculate NDUI
ndui_c = (ds_dmsp_interp_c - ds_ndvi_c_)/(ds_dmsp_interp_c + ds_ndvi_c_)
ndui_c[ndui_c > 1.0] = 1.0
ndui_c[ndui_c < -1.0] = -1.0

# Stack the result and save as NetCDF for one city and one year
ds_ndui_c = xr.Dataset({
    'ndui': xr.DataArray(
        data=ndui_c,  # enter data here
        dims=['lat', 'lon'],
        coords={'lat': ds_ndvi_c.y.values, 'lon': ds_ndvi_c.x.values},
    ),
})

print(ds_ndui_c)

# Save the NetCDF file
ds_ndui_c.to_netcdf('ndui_' + year + '_' + city[0] + '.nc')

Downloading:   0%|          | 0.00/5.25M [00:00<?, ?iB/s]

<xarray.Dataset> Size: 18MB
Dimensions:  (lat: 1486, lon: 1485)
Coordinates:
  * lat      (lat) float64 12kB 28.9 28.9 28.9 28.9 28.9 ... 28.5 28.5 28.5 28.5
  * lon      (lon) float64 12kB 76.9 76.9 76.9 76.9 76.9 ... 77.3 77.3 77.3 77.3
Data variables:
    ndui     (lat, lon) float64 18MB -0.4356 -0.4527 -0.4538 ... 0.7044 0.7053


In [4]:
!ls

drive  ndui_2022_delhi.nc  sample_data


In [5]:
import ee
import pandas as pd
import xarray as xr

def generate_ndui_for_city(city_name, lat, lon, year=2022):
    """
    Generates the NDUI for a specific city and year and saves it as a NetCDF file.

    Parameters:
    city_name (str): The name of the city.
    lat (float): Latitude of the city.
    lon (float): Longitude of the city.
    year (int): The year for which to generate the NDUI (default: 2022).
    """

    # Convert year to string
    year_str = str(year)

    # Define the AOI for the city
    lats, late = lat - 0.2, lat + 0.2
    lons, lone = lon - 0.2, lon + 0.2
    aoi = ee.Geometry.Polygon(
          [[[lons, lats],
            [lone, lats],
            [lone, late],
            [lons, late]]])

    # Load the Landsat 7 dataset for the year
    L7 = ee.ImageCollection("LANDSAT/LE07/C02/T1_TOA").filterDate(year_str + '-01-01', year_str + '-12-31')

    def fun4(img):
        bad1 = img.select('B1').eq(0.0)
        bad2 = img.select('B2').eq(0.0)
        bad3 = img.select('B3').eq(0.0)
        bad4 = img.select('B4').eq(0.0)
        bad5 = img.select('B5').eq(0.0)
        bad7 = img.select('B7').eq(0.0)
        mask = img.mask().And(bad1.Or(bad2).Or(bad3).Or(bad4).Or(bad5).Or(bad7).Not())
        masked = img.mask(mask)
        ndvi = masked.normalizedDifference(["B4", "B3"])
        return ndvi

    NDVIs = L7.map(fun4)

    # Generate NDVI statistics for the year
    Mean_NDVI = NDVIs.median()
    Max_NDVI = NDVIs.max()
    Min_NDVI = NDVIs.min()

    mosaic = Mean_NDVI.where(Max_NDVI.gt(0.4), Max_NDVI)
    mosaic = mosaic.where(Min_NDVI.lt(-0.2), Min_NDVI)

    mosaic = mosaic.set('system:time_start', 0)
    ds_ndvi_c = mosaic.wx.to_xarray(region=aoi.bounds(), scale=30)
    ds_ndvi_c_ = ds_ndvi_c.nd.values[0,:,:]

    # Load the DMSP dataset and interpolate for the same year
    ds_dmsp_1992_2022_city = xr.open_dataset(f'/content/drive/MyDrive/ndui/dmsp_swinIR_1992_2022_{city_name}.nc')
    ds_dmsp_interp_c = ds_dmsp_1992_2022_city.sel(time=slice(year_str, year_str)).interp(lon=ds_ndvi_c.x, lat=ds_ndvi_c.y).dmsp.values[0,:,:] / 63.0

    # Calculate NDUI
    ndui_c = (ds_dmsp_interp_c - ds_ndvi_c_) / (ds_dmsp_interp_c + ds_ndvi_c_)
    ndui_c[ndui_c > 1.0] = 1.0
    ndui_c[ndui_c < -1.0] = -1.0

    # Create Dataset and save as NetCDF
    ds_ndui_c = xr.Dataset({
        'ndui': xr.DataArray(
            data=ndui_c,
            dims=['lat', 'lon'],
            coords={'lat': ds_ndvi_c.y.values, 'lon': ds_ndvi_c.x.values},
        ),
    })

    print(f"Generated NDUI for {city_name} in {year}.")
    ds_ndui_c.to_netcdf(f'ndui_{year_str}_{city_name}.nc')

    # # Save the file to the drive
    # !mv ndui_*.nc /content/drive/MyDrive/ndui

In [6]:
# List of cities and their coordinates
cities = ['delhi', 'hongkong', 'melbourne', 'dubai', 'london', 'berlin', 'tokyo', 'dhaka', 'shanghai', 'saopaulo', 'mexicocity', 'cairo']
lats_  = [28.7041, 22.3193, -37.8136, 25.2048, 51.5072, 52.5200, 35.6764, 23.8041, 31.2304, -23.5558, 19.4326, 30.0444]
lons_  = [77.1025, 114.1694, 144.9631, 55.2708, -0.1276, 13.4050, 139.6500, 90.4152, 121.4737, -46.6396, -99.1332, 31.2357]

# Loop through each city and generate NDUI for 2022
for city, lat, lon in zip(cities, lats_, lons_):
    generate_ndui_for_city(city, lat, lon, year=2022)

Downloading:   0%|          | 0.00/5.25M [00:00<?, ?iB/s]

Generated NDUI for delhi in 2022.


Downloading:   0%|          | 0.00/4.20M [00:00<?, ?iB/s]

Generated NDUI for hongkong in 2022.


Downloading:   0%|          | 0.00/4.68M [00:00<?, ?iB/s]

...

Generated NDUI for dubai in 2022.


Downloading:   0%|          | 0.00/4.93M [00:00<?, ?iB/s]

Generated NDUI for london in 2022.


Downloading:   0%|          | 0.00/3.95M [00:00<?, ?iB/s]

FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = '/content/drive/MyDrive/ndui/dmsp_swinIR_1992_2022_berlin.nc', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [7]:
!cp ndui_*.nc /content/drive/MyDrive/ndui

In [9]:
!ls /content/drive/MyDrive/ndui/dmsp_swinIR_1992_2022_*.nc

/content/drive/MyDrive/ndui/dmsp_swinIR_1992_2022_delhi.nc
/content/drive/MyDrive/ndui/dmsp_swinIR_1992_2022_dubai.nc
/content/drive/MyDrive/ndui/dmsp_swinIR_1992_2022_hongkong.nc
/content/drive/MyDrive/ndui/dmsp_swinIR_1992_2022_london.nc
/content/drive/MyDrive/ndui/dmsp_swinIR_1992_2022_melbourne.nc
